In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os, glob, shutil
import torch, torch.nn as nn, torch.nn.functional as F
import cv2 as cv, numpy as np
from collections import deque
import random, yaml
from ultralytics import YOLO

In [3]:
# 당신의 데이터 최상위 폴더(여기 아래 어딘가에 클래스 폴더 6개가 있음)
RAW_ROOT = r"trashnet/data/dataset-resized"

# 1) 맥 압축 잔여물 제거
def clean_mac_artifacts(root: str):
    for d in glob.glob(os.path.join(root, "**", "__MACOSX"), recursive=True):
        shutil.rmtree(d, ignore_errors=True)
    for p in glob.glob(os.path.join(root, "**", "._*"), recursive=True):
        try: os.remove(p)
        except: pass
    for p in glob.glob(os.path.join(root, "**", ".DS_Store"), recursive=True):
        try: os.remove(p)
        except: pass

clean_mac_artifacts(RAW_ROOT)

# 2) 클래스 폴더 깊이 자동 탐지 (cardboard/glass/metal/paper/plastic/trash 찾기)
def find_class_root(root: str):
    expected = {"cardboard","glass","metal","paper","plastic","trash"}
    names = {n for n in os.listdir(root) if os.path.isdir(os.path.join(root,n))}
    # 현재 깊이에 클래스 폴더가 보이면 OK
    if len(expected & names) >= 4:
        return root
    # 한 단계 더 들어가서 확인
    for n in names:
        sub = os.path.join(root, n)
        if not os.path.isdir(sub): 
            continue
        sub_names = {m for m in os.listdir(sub) if os.path.isdir(os.path.join(sub,m))}
        if len(expected & sub_names) >= 4:
            return sub
    return root  # 못 찾으면 원래 root 반환(추후 프린트로 확인)

DATA_DIR = find_class_root(RAW_ROOT)
print("사용할 DATA_DIR:", os.path.abspath(DATA_DIR))
print("하위 폴더 샘플:", os.listdir(DATA_DIR)[:10])

사용할 DATA_DIR: c:\Users\jk316\practice\trashnet\data\dataset-resized\dataset-resized
하위 폴더 샘플: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [4]:
SRC = DATA_DIR  # 위에서 찾은 클래스 폴더 루트
DST = r"data/trash6"  # 새롭게 만들 탐지 데이터셋 루트
CLASSES = ["cardboard","glass","metal","paper","plastic","trash"]
img_ext = {".jpg",".jpeg",".png",".bmp"}
train_ratio = 0.8

# 깨끗이 생성
if os.path.exists(DST): shutil.rmtree(DST)
for p in ["images/train","images/val","labels/train","labels/val"]:
    os.makedirs(os.path.join(DST, p), exist_ok=True)

def all_images(cls):
    p = os.path.join(SRC, cls)
    return [f for f in glob.glob(os.path.join(p, "*")) if os.path.splitext(f)[1].lower() in img_ext]

for cid, cls in enumerate(CLASSES):
    imgs = all_images(cls)
    random.shuffle(imgs)
    n_tr = int(len(imgs)*train_ratio)
    splits = [("train", imgs[:n_tr]), ("val", imgs[n_tr:])]
    for split, arr in splits:
        for src in arr:
            name = f"{cls}_{os.path.splitext(os.path.basename(src))[0]}"
            img_dst = os.path.join(DST, f"images/{split}", name + os.path.splitext(src)[1].lower())
            lbl_dst = os.path.join(DST, f"labels/{split}", name + ".txt")
            shutil.copy2(src, img_dst)
            # 풀프레임 박스: cx=0.5, cy=0.5, w=1.0, h=1.0
            with open(lbl_dst, "w") as f:
                f.write(f"{cid} 0.5 0.5 1.0 1.0\n")

yaml_path = os.path.join(DST, "trash6.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.safe_dump({
        "path": DST,
        "train": "images/train",
        "val": "images/val",
        "names": CLASSES
    }, f, sort_keys=False, allow_unicode=True)

print("변환 완료:", DST, "\nYAML:", yaml_path)


변환 완료: data/trash6 
YAML: data/trash6\trash6.yaml


In [5]:
det_model = YOLO("yolov8n.pt")
det_model.train(data=os.path.join(DST, "trash6.yaml"), epochs=30, imgsz=640, batch=16)


Ultralytics 8.3.203  Python-3.11.13 torch-2.5.1 CPU (Intel Core Ultra 7 258V)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/trash6\trash6.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000018742EB6550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [9]:
# det_model = YOLO("runs/detect/train/weights/best.pt")
# 학습 직후면 det_model 그대로 사용 가능
cap = cv.VideoCapture(0)
assert cap.isOpened(), "웹캠을 열 수 없습니다."

while True:
    ok, frame = cap.read()
    if not ok: break
    results = det_model.predict(source=frame, imgsz=640, conf=0.25, verbose=False)
    annotated = results[0].plot()  # 박스/라벨/점수 그려진 프레임
    cv.imshow("YOLOv8-Detection (full-frame bbox)", annotated)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
